In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv("hardware.csv")
todrop = ["Website", "Description", "Location", "City",
          "State", "Continent", "Keywords", "Date Added", "Investors", "Name"
         ]
data.drop(todrop, axis=1, inplace=True)
tonumeric = ['Employee Count Last Month', 'Employee Count 6 Months Ago', 
            'Employees Added Last 6 Months',
            'Est. Monthly Uniques: The estimated monthly unique visitors to the company’s website.', 
            'Est. Monthly Uniques Week Ago: The estimated monthly unique visitors to the company’s website during the past 7 days.', 
            'Est. Monthly Uniques Month Ago', 
            'Employees Added This Month', 
            "Employee Count: The number of total employees a company has reported on it's annual filing.", 
            "New Funding Employee Growth", 
            'Est. Monthly Mobile Downloads Month Ago: The estimated number of monthly mobile downloads last month', 
            "Est. Monthly Mobile Downloads Week Ago: The estimated monthly mobile downloads one week ago from the U.S. iTunes app store.", 
            'Est. Mobile Downloads: The estimated monthly mobile downloads from the U.S. iTunes app store.'
            ]

ridstring = ['Employee Growth Last 6 Months', 'Last Funding Amount',
             'Employee Growth Last Mo: Growth of the number of total employees a company', 
             'Monthly Uniques Week Over Week Growth: The percent change in the estimated monthly unique visitors to the company’s website in the past month.', 
             'Monthly Uniques Month Over Month Growth', 
             'Monthly Mobile Downloads Week Over Week Growth: The change in the estimated monthly mobile downloads from the U.S. iTunes app store during the past week.', 
             'Monthly Mobile Downloads Month Over Month Growth: The change in the estimated monthly mobile downloads from the U.S. iTunes app store from the past month.']

ridstringextra = ['Total Funding']

trantab = str.maketrans({"$" : None, "," : None, "M" : None, "B" : None, "." : None, "%": None})
riddollarcomma = lambda x : str(x).translate(trantab)

def work(x):
    s = str(x)
    if "M" in s:
        if "." in s:
            return s.translate(trantab) + "00000"
        return s.translate(trantab) + "000000"
    if "B" in s:
        if "." in s:
            return s.translate(trantab) + "00000000"
        return s.translate(trantab) + "000000000"
    return s.translate(trantab)
        

rid = lambda x : str(x)[:len(str(x)) - 1]


for string in tonumeric:
    data.loc[:, string] = pd.to_numeric(data[string], errors='coerce')
    
for string in ridstring:
    data.loc[:, string] = pd.to_numeric(data[string].apply(riddollarcomma), errors='coerce')

for string in ridstringextra:
    data.loc[:, string] = pd.to_numeric(data[string].apply(work), errors='coerce')

data.loc[:, "Last Funding"] = pd.to_datetime(data["Last Funding"])
    
pd.get_dummies(data)

,Growth Score: default ranking for all companies in Mattermark. It measures how quickly a company is gaining traction at a given point in time.,Mindshare Score: quickly identify companies whose online footprint is growing the fastest. It is measured on a weekly basis and averaged over time to provide a trending metric over time.,"Weekly Momentum: the calculation of web, mobile, and social traction for a company on a weekly basis. It is similar to the Mindshare Score in that they both use the same metrics to measure traction.",Employee Count: The number of total employees a company has reported on it's annual filing.,Employee Count Last Month,Employees Added This Month,Employee Growth Last Mo: Growth of the number of total employees a company,Employee Count 6 Months Ago,Employees Added Last 6 Months,Employee Growth Last 6 Months,...,"Industries_hardware,robotics","Industries_hardware,semiconductors","Industries_hardware,software development","Industries_hardware,transportation",Has Mobile App_No,Has Mobile App_Yes,Has Google Play App_No,Has Google Play App_Yes,Has iTunes App_No,Has iTunes App_Yes
0,1041,-23,-49,562.0,370.0,192.0,52.0,242.0,320.0,132.0,...,0,0,0,0,0,1,0,1,0,1
1,978,764,-423,158.0,148.0,10.0,7.0,114.0,44.0,39.0,...,0,0,0,0,0,1,0,1,0,1
2,840,558,1431,145.0,67.0,78.0,116.0,33.0,112.0,339.0,...,0,0,0,0,1,0,1,0,1,0
3,817,649,0,139.0,140.0,-1.0,-1.0,127.0,12.0,9.0,...,0,0,0,0,0,1,0,1,0,1
4,812,317,0,NaN,994.0,22.0,2.0,934.0,82.0,9.0,...,0,0,0,0,1,0,1,0,1,0
5,717,113,0,340.0,322.0,18.0,6.0,228.0,112.0,49.0,...,0,0,0,0,1,0,1,0,1,0
6,712,190,0,319.0,314.0,5.0,2.0,291.0,28.0,10.0,...,0,0,0,0,1,0,1,0,1,0
7,679,390,0,109.0,107.0,2.0,2.0,78.0,31.0,40.0,...,0,0,0,0,1,0,1,0,1,0
8,670,-99,281,193.0,181.0,12.0,7.0,121.0,72.0,60.0,...,0,0,0,0,1,0,1,0,1,0
9,664,-71,0,436.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,1,0,1,0,1,0


In [2]:
allthings = []
manythings = [thing.split(",") for thing in list(data['Industries'])]
for listthing in manythings:
    for thing in listthing:
        if thing not in allthings:
            allthings.append(thing)
allthings
for industry in allthings:
    indices = np.arange(1, 901)
    onehot = np.zeros(900)
    for i in range(900):
        if industry in manythings[i]:
            onehot[i] = 1
    newseries = pd.Series(onehot, indices)
    data.loc[:,industry] = newseries
    

In [3]:
def killnan(x):
    if type(x) == float:
        return "None"
    else:
        return x

In [4]:
allmodels = []
businessModels = [thing.split(",") for thing in list(data['Business Models'].apply(killnan))]
for listthing in businessModels:
    for thing in listthing:
        if thing not in allmodels:
            allmodels.append(thing)
for model in allmodels:
    indices = np.arange(1, 901)
    onehot = np.zeros(900)
    for i in range(900):
        if model in businessModels[i]:
            onehot[i] = 1
    newseries = pd.Series(onehot, indices)
    data.loc[:, model] = newseries
data.drop(["Industries", "Country", "Business Models"], axis=1, inplace=True)
list(pd.get_dummies(data.dropna(axis=1, thresh=600)))

['Growth Score: default ranking for all companies in Mattermark. It measures how quickly a company is gaining traction at a given point in time.',
 'Mindshare Score: quickly identify companies whose online footprint is growing the fastest. It is measured on a weekly basis and averaged over time to provide a trending metric over time.',
 'Weekly Momentum: the calculation of web, mobile, and social traction for a company on a weekly basis. It is similar to the Mindshare Score in that they both use the same metrics to measure traction. ',
 "Employee Count: The number of total employees a company has reported on it's annual filing.",
 'Employee Count Last Month',
 'Employees Added This Month',
 'Employee Growth Last Mo: Growth of the number of total employees a company',
 'Employees Added Last 6 Months',
 'Employee Growth Last 6 Months',
 'Monthly Uniques Week Over Week Growth: The percent change in the estimated monthly unique visitors to the company’s website in the past month.',
 'Month

In [5]:
dataset = pd.get_dummies(data.dropna(axis=1, thresh=600)).fillna(value=0)
names = ['Growth Score: default ranking for all companies in Mattermark. It measures how quickly a company is gaining traction at a given point in time.', 
        'Mindshare Score: quickly identify companies whose online footprint is growing the fastest. It is measured on a weekly basis and averaged over time to provide a trending metric over time.']
growth = dataset[names[0]].values
mind = dataset[names[1]].values


from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit(dataset.drop(names, axis=1).values, growth)


/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/scipy/linalg/basic.py:884: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [6]:
np.savetxt("cleaneddata.csv", dataset, fmt='%i', delimiter=',')

In [ ]:
from sklearn.svm import SVR
import matplotlib.pyplot as plt

traininggrowth = growth[:600]
trainingmind = mind[:600]
trainingdata = dataset.drop(names, axis=1).values[:600]

valgrowth = growth[600:]
valmind = mind[600:]
valdata = dataset.drop(names, axis=1).values[600:]

svr_lin = SVR(kernel='linear', C=1e3)
svr_lin.fit(trainingdata, traininggrowth)
trainresults = svr_lin.predict(trainingdata)
valresults = svr_lin.predict(valdata)

traininggrowth - trainresults
#print(np.linalg.norm(valgrowth - valresults) / 300)
